In [1]:
#imports
import numpy as np
import pandas as pd
from tqdm import tqdm
import scipy.signal as signal #signal processing stuff (e.g. filters, hilbert transform, etc.)
from scipy.stats import linregress
import scipy
import struct
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import svgutils.transform as sg
import sys

#nelpy!
import nelpy as nel  #recommended import for nelpy
import nelpy.plotting as npl  #recommended import for the nelpy plotting library
import nelpy.io.trodes as neltro

#I saved my online ripple detection filter taps and stuff in a file....gonna import this for all analysis
import imp
ripple_filtering = imp.load_source('ripple_filtering', '/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/DataAnalysisScripts/ripple_filtering.py')

In [2]:
#paper plot settings
sns.set(rc={'figure.figsize': (12, 4),'lines.linewidth': 1, 'font.size': 18, 'axes.labelsize': 16, 'axes.titlesize':18, 'legend.fontsize': 12, 'ytick.labelsize': 12, 'xtick.labelsize': 12 })
sns.set_style('white')
sns.set_color_codes(palette='colorblind')
#plots show up within jupyter for matplotlib
%matplotlib inline 

In [3]:
FS=3000
FS_system=30000

In [4]:
filePath = "/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChan.LFP/singleChan.timestamps.dat"
print("*****************Loading LFP Timestamps*****************")
f = open(filePath,'rb')
instr = f.readline()
while (instr != b'<End settings>\n') :
   print(instr)
   instr = f.readline()
print('Current file position', f.tell())
timeStamps = np.fromfile(f, dtype=np.uint32)
timeStampsSeconds = (timeStamps-timeStamps[0])/FS_system
print("Done!")

filePath = "/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChan.LFP/singleChan.LFP_nt2ch1.dat" 
print("*****************Loading LFP Data*****************")
f = open(filePath,'rb')
instr = f.readline()
while (instr != b'<End settings>\n') :
   print(instr)
   instr = f.readline()
print('Current file position', f.tell())
dataT2 = np.fromfile(f, dtype=np.int16)

filePath = "/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChan.LFP/singleChan.LFP_nt3ch1.dat"
print("*****************Loading LFP Data*****************")
f = open(filePath,'rb')
instr = f.readline()
while (instr != b'<End settings>\n') :
   print(instr)
   instr = f.readline()
print('Current file position', f.tell())
dataT3 = np.fromfile(f, dtype=np.int16)

filePath = "/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChan.LFP/singleChan.LFP_nt4ch1.dat"
print("*****************Loading LFP Data*****************")
f = open(filePath,'rb')
instr = f.readline()
while (instr != b'<End settings>\n') :
   print(instr)
   instr = f.readline()
print('Current file position', f.tell())
dataT4 = np.fromfile(f, dtype=np.int16)

#Decimate
start = 0
while(timeStamps[start]%10 != 0):
    start+=1
timeStamps = timeStamps[start::10]
dataT2 = dataT2[start::10]
dataT3 = dataT3[start::10]
dataT4 = dataT4[start::10]
timeStampsSeconds = timeStampsSeconds[start::10]
print(timeStamps[0])
print("Data decimated to match data sent to modules")

*****************Loading LFP Timestamps*****************
b'<Start settings>\n'
b'Description: LFP timestamps\n'
b'Byte_order: little endian\n'
b'Original_file: singleChan.rec\n'
b'Clock rate: 30000\n'
b'Decimation: 1\n'
b'Time_offset: 0\n'
b'Fields: <time uint32>\n'
Current file position 185
Done!
*****************Loading LFP Data*****************
b'<Start settings>\n'
b'Description: LFP data for one channel\n'
b'Byte_order: little endian\n'
b'Original_file: singleChan.rec\n'
b'nTrode_ID: 2\n'
b'nTrode_channel: 1\n'
b'Clock rate: 30000\n'
b'Voltage_scaling: 0.195\n'
b'Decimation: 1\n'
b'First_timestamp: 19935\n'
b'Reference: on\n'
b'Low_pass_filter: 400\n'
b'Fields: <voltage int16>\n'
Current file position 294
*****************Loading LFP Data*****************
b'<Start settings>\n'
b'Description: LFP data for one channel\n'
b'Byte_order: little endian\n'
b'Original_file: singleChan.rec\n'
b'nTrode_ID: 3\n'
b'nTrode_channel: 1\n'
b'Clock rate: 30000\n'
b'Voltage_scaling: 0.195\n'
b'Deci

In [11]:
#Only run first time for data generation! This takes a while otherwise
# smoothed_envelope3, rippleData3 = ripple_filtering.rippleBandFilterSimulated(dataT3, timeStampsSeconds, FS,ripple_filtering.bandpassFilterTaps,ripple_filtering.lowpassFilterTaps)
# smoothed_envelope4, rippleData4 = ripple_filtering.rippleBandFilterSimulated(dataT4, timeStampsSeconds, FS,ripple_filtering.bandpassFilterTaps,ripple_filtering.lowpassFilterTaps)
# np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/smoothed_envelope_simulatedT3.out",smoothed_envelope3,fmt='%10.5f')
# np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/smoothed_envelope_simulatedT4.out",smoothed_envelope4,fmt='%10.5f')
# smoothed_envelope2 = np.loadtxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/smoothed_envelopeT2.out")
# smoothed_envelope3, rippleData3 = ripple_filtering.rippleBandFilter(dataT3, timeStampsSeconds, FS=FS)
# smoothed_envelope4, rippleData4 = ripple_filtering.rippleBandFilter(dataT4, timeStampsSeconds, FS=FS)
# np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/smoothed_envelopeT3.out",smoothed_envelope3,fmt='%10.5f')
# np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/smoothed_envelopeT4.out",smoothed_envelope4,fmt='%10.5f')

In [15]:
np.savez_compressed("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/figureData/Figure6/smoothed_envelopeT2.npz",smoothed_envelopeT2=smoothed_envelopeT2)
np.savez_compressed("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/figureData/Figure6/smoothed_envelopeT3.npz",smoothed_envelopeT3=smoothed_envelopeT3)
np.savez_compressed("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/figureData/Figure6/smoothed_envelopeT4.npz",smoothed_envelopeT4=smoothed_envelopeT4)

In [16]:
smoothed_envelopeT2 = np.load("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/figureData/Figure6/smoothed_envelopeT2.npz")['smoothed_envelopeT2']
smoothed_envelopeT3 = np.load("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/figureData/Figure6/smoothed_envelopeT3.npz")['smoothed_envelopeT3']
smoothed_envelopeT4 = np.load("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/figureData/Figure6/smoothed_envelopeT4.npz")['smoothed_envelopeT4']

In [17]:
#set threshold, find threshold crossings, and find ripple bounds
SecondaryThresholdSigma = 0
thresholdSigma = 3
lengthCriteria = 0.015 #15 ms

In [18]:
#Tetrode 2
thresholdT2 = np.mean(smoothed_envelopeT2) + thresholdSigma*np.std(smoothed_envelopeT2)

#rippleEvents contains indices of events in smoothed_envelope that meet threshold
rippleEventsT2, _ = ripple_filtering.thresholdCrossings(smoothed_envelopeT2, thresholdT2)
#remove events that don't meet length criteria
rippleEventsT2 = rippleEventsT2[rippleEventsT2[:,1] - rippleEventsT2[:,0] >= np.round(FS * lengthCriteria),:]

#find start and end of putative ripple aka when ripple band signal comes back down to mean
secondaryThresholdT2 = np.mean(smoothed_envelopeT2) + SecondaryThresholdSigma*np.std(smoothed_envelopeT2)
rippleBoundsT2, broaderMaxesT2 = ripple_filtering.thresholdCrossings(smoothed_envelopeT2, secondaryThresholdT2)

#find windows that match and where they match aka where in the larger bound window does the smaller bound fit
outerBoundaryIndicesT2 = np.searchsorted(rippleBoundsT2[:,0], rippleEventsT2[:,0])
outerBoundaryIndicesT2 = outerBoundaryIndicesT2 - 1 #subtract 1 since searchsorted returns index after where it belongs

#Find extended boundaries for ripple events by pairing to largere window
#   (Note that there may be repeats if the larger window contains multiple > 3SD sections)
rippleBoundsT2 = rippleBoundsT2[outerBoundaryIndicesT2,:]
rippleMaxesT2 = broaderMaxesT2[outerBoundaryIndicesT2]

# Now, since all that we care about are the larger windows, so we should get rid of repeats
_, unique_idxT2 = np.unique(rippleBoundsT2[:,0], return_index=True)
rippleBoundsT2 = rippleBoundsT2[unique_idxT2,:]
rippleMaxesT2 = rippleMaxesT2[unique_idxT2]
rippleEventsT2 = rippleEventsT2[unique_idxT2,:]

offlineRippleDetectionsT2 = np.zeros(smoothed_envelopeT2.size)
for i in range(0, rippleMaxesT2.size):
    offlineRippleDetectionsT2[(rippleBoundsT2[i][0]):(rippleBoundsT2[i][1])] = rippleMaxesT2[i]

In [19]:
#Tetrode 3
thresholdT3 = np.mean(smoothed_envelopeT3) + thresholdSigma*np.std(smoothed_envelopeT3)

#rippleEvents contains indices of events in smoothed_envelope that meet threshold
rippleEventsT3, _ = ripple_filtering.thresholdCrossings(smoothed_envelopeT3, thresholdT3)
#remove events that don't meet length criteria
rippleEventsT3 = rippleEventsT3[rippleEventsT3[:,1] - rippleEventsT3[:,0] >= np.round(FS * lengthCriteria),:]

#find start and end of putative ripple aka when ripple band signal comes back down to mean
secondaryThresholdT3 = np.mean(smoothed_envelopeT3) + SecondaryThresholdSigma*np.std(smoothed_envelopeT3)
rippleBoundsT3, broaderMaxesT3 = ripple_filtering.thresholdCrossings(smoothed_envelopeT3, secondaryThresholdT3)

#find windows that match and where they match aka where in the larger bound window does the smaller bound fit
outerBoundaryIndicesT3 = np.searchsorted(rippleBoundsT3[:,0], rippleEventsT3[:,0])
outerBoundaryIndicesT3 = outerBoundaryIndicesT3- 1 #subtract 1 since searchsorted returns index after where it belongs

#Find extended boundaries for ripple events by pairing to largere window
#   (Note that there may be repeats if the larger window contains multiple > 3SD sections)
rippleBoundsT3 = rippleBoundsT3[outerBoundaryIndicesT3,:]
rippleMaxesT3 = broaderMaxesT3[outerBoundaryIndicesT3]

# Now, since all that we care about are the larger windows, so we should get rid of repeats
_, unique_idxT3 = np.unique(rippleBoundsT3[:,0], return_index=True)
rippleBoundsT3 = rippleBoundsT3[unique_idxT3,:]
rippleMaxesT3 = rippleMaxesT3[unique_idxT3]
rippleEventsT3 = rippleEventsT3[unique_idxT3,:]

offlineRippleDetectionsT3 = np.zeros(smoothed_envelopeT3.size)
for i in range(0, rippleMaxesT3.size):
    offlineRippleDetectionsT3[(rippleBoundsT3[i][0]):(rippleBoundsT3[i][1])] = rippleMaxesT3[i]

In [42]:
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/rippleBoundsStartT2.out",rippleBoundsT2[:,0],fmt='%10.5f')
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/rippleBoundsEndT2.out",rippleBoundsT2[:,1],fmt='%10.5f')
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/rippleBoundsStartT3.out",rippleBoundsT3[:,0],fmt='%10.5f')
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/rippleBoundsEndT3.out",rippleBoundsT3[:,1],fmt='%10.5f')

In [44]:
#merge t2 and t3 detections for canonical ripple
#merge t2 and t3 detections for canonical ripple
#ripples are defined as intersection with two channels. start is the earliest start time out of the two channnels
#end is the latter end of the two channels. This should give a conservative detection latency estimate while
#encapsulating the entire ripple event across channels
rippleBounds = []
for i in range(0, rippleMaxesT2.size):
    moo = np.squeeze(np.where(offlineRippleDetectionsT3[rippleBoundsT2[i][0]:rippleBoundsT2[i][1]] > 0))
    if(moo.size > 0):
        if(offlineRippleDetectionsT3[rippleBoundsT2[i][0]] > 0):
            rippleBounds.append([rippleBoundsT2[i][0], rippleBoundsT2[i][1]])
        else:
            if(moo.size == 1): 
                index = np.squeeze(np.where(moo+rippleBoundsT2[i][0] == rippleBoundsT3[:,0]))
                rippleBounds.append([rippleBoundsT3[index][0],rippleBoundsT3[index][1]])
            else:
                index = np.squeeze(np.where(moo[0]+rippleBoundsT2[i][0] == rippleBoundsT3[:,0]))
                rippleBounds.append([rippleBoundsT3[index][0],rippleBoundsT3[index][1]])

rippleBounds = np.asarray(rippleBounds)

#Merge union of ripple detections
offlineRippleDetections = np.zeros(smoothed_envelopeT3.size)
for i in range(0, rippleBounds.shape[0]):
    offlineRippleDetections[(rippleBounds[i][0]):(rippleBounds[i][1])] = rippleMaxesT2[i]

In [45]:
rippleBounds[0:10]

array([[ 61741,  62979],
       [ 76214,  76842],
       [ 93610,  94898],
       [102885, 103749],
       [105662, 106174],
       [110621, 110955],
       [111693, 111937],
       [112006, 112493],
       [112712, 112886],
       [125374, 126217]])

<div class="alert alert-info">
The comment in cell 20 can easily be implemented in python but I just don't want to think and would rather use a nested loop...so it's gonna be done within the C++ functions voting two channel function! It just makes life easier. I think I have code that does the merging with `nelpy` but this is faster for me to prototype :)
</div>

# Call C++ analysis function
Compile with below! Note: I just like having all warnings shown really no reason to compile with those flags<br>
`g++ -o votingTwoOfThree -Wall -pedantic -std=c++11 -pthread voting2of3.cpp`<br> 
or <br>
`g++ -o votingTwoOfTwo -Wall -pedantic -std=c++11 -pthread voting2of2.cpp` <--- only one used in the paper<br>
*make sure single channel is compiled as well with below! Run it only after the two channel begins i*<br>
`g++ -o twoChanDefnSingleChanAnalysis -Wall -pedantic -std=c++11 -pthread singleChanAnalysis_twoChanDefn.cpp`

<div class="alert alert-warning">
The code here is changed to compile metrics for each one of these. File names have been changed for convenience...compile appropriate .cpp file. The file structure will need to be changed if anyone other than me is running this code
</div>